In [ ]:
from flask import Flask, request, jsonify
import time
import os
import azure.cognitiveservices.speech as speechsdk
import json 
from datetime import datetime 
import subprocess
from pydub.utils import mediainfo


In [1]:
from flask import Flask, request, jsonify
import time
import os
import azure.cognitiveservices.speech as speechsdk
import json 
from datetime import datetime 
import subprocess
from pydub.utils import mediainfo
import json
import cv2
import os, shutil
import boto3
from io import StringIO
import sys
import time


class VideoDetect:
    jobId = ''
    rek = boto3.client('rekognition', region_name='ap-south-1')
    sqs = boto3.client('sqs', region_name='ap-south-1')
    sns = boto3.client('sns', region_name='ap-south-1')

    roleArn = ''
    bucket = ''
    video = ''
    startJobId = ''

    sqsQueueUrl = ''
    snsTopicArn = ''
    processType = ''

    def __init__(self, role, bucket, video, segment):
        self.roleArn = role
        self.bucket = bucket
        self.video = video
        self.segment = segment

    def GetSQSMessageSuccess(self):

        jobFound = False
        succeeded = False

        dotLine=0
        while jobFound == False:
            sqsResponse = self.sqs.receive_message(QueueUrl=self.sqsQueueUrl, MessageAttributeNames=['ALL'],
                                                   MaxNumberOfMessages=10)

            if sqsResponse:

                if 'Messages' not in sqsResponse:
                    if dotLine<40:
                        print('.', end='')
                        dotLine=dotLine+1
                    else:
                        print()
                        dotLine=0
                    sys.stdout.flush()
                    time.sleep(5)
                    continue

                for message in sqsResponse['Messages']:
                    notification = json.loads(message['Body'])
                    rekMessage = json.loads(notification['Message'])
                    print(rekMessage['JobId'])
                    print(rekMessage['Status'])
                    if rekMessage['JobId'] == self.startJobId:
                        # print('Matching Job Found:' + rekMessage['JobId'])
                        jobFound = True
                        if (rekMessage['Status']=='SUCCEEDED'):
                            succeeded=True

                        self.sqs.delete_message(QueueUrl=self.sqsQueueUrl,
                                                ReceiptHandle=message['ReceiptHandle'])
                    else:
                        pass
                        # print("Job didn't match:" +
                        #       str(rekMessage['JobId']) + ' : ' + self.startJobId)
                    # Delete the unknown message. Consider sending to dead letter queue
                    self.sqs.delete_message(QueueUrl=self.sqsQueueUrl,
                                            ReceiptHandle=message['ReceiptHandle'])


        return succeeded

    # def StartCelebrityRecognition(self):
    def StartContentModeration(self):
        # response=self.rek.start_label_detection(Video={'S3Object': {'Bucket': self.bucket, 'Name': self.video}},
        #                                         NotificationChannel={'RoleArn': self.roleArn, 'SNSTopicArn': self.snsTopicArn})
        # response=self.rek.start_celebrity_recognition(Video={'S3Object': {'Bucket': self.bucket, 'Name': self.video}},
        print(self.bucket)
        print(self.video)
        response=self.rek.start_content_moderation(Video={'S3Object': {'Bucket': self.bucket, 'Name': self.video}},
                                                   NotificationChannel={'RoleArn': self.roleArn, 'SNSTopicArn': self.snsTopicArn})

        self.startJobId=response['JobId']
        # print('Start Job Id: ' + self.startJobId)
        
    def StartCelebrityRecognition(self):
        # response=self.rek.start_label_detection(Video={'S3Object': {'Bucket': self.bucket, 'Name': self.video}},
        #                                         NotificationChannel={'RoleArn': self.roleArn, 'SNSTopicArn': self.snsTopicArn})
        # response=self.rek.start_celebrity_recognition(Video={'S3Object': {'Bucket': self.bucket, 'Name': self.video}},
        print(self.bucket)
        print(self.video)
        response=self.rek.start_celebrity_recognition(Video={'S3Object': {'Bucket': self.bucket, 'Name': self.video}},
                                                   NotificationChannel={'RoleArn': self.roleArn, 'SNSTopicArn': self.snsTopicArn})

        self.startJobId=response['JobId']
        # print('Start Job Id: ' + self.startJobId)

    # def GetCelebrityRecognitionResults(self):
    def GetContentModerationResults(self):
        maxResults = 100
        paginationToken = ''
        # finished = False
        #
        # while finished == False:
        response = self.rek.get_content_moderation(JobId=self.startJobId,
                                                   MaxResults=maxResults,
                                                   NextToken=paginationToken,
                                                   SortBy='TIMESTAMP')

        print('Codec: ' + response['VideoMetadata']['Codec'])
        # print('Duration: ' + str(response['VideoMetadata']['DurationMillis']))
        print('Format: ' + response['VideoMetadata']['Format'])
        print('Frame rate: ' + str(response['VideoMetadata']['FrameRate']))
        print()

        result_list = []

        for moderation_label_info in response['ModerationLabels']:
            moderation_label_dict = {}
            segment_offset_seconds = (self.segment - 1) * 60000
            timestamp = moderation_label_info["Timestamp"]
            timestamp += segment_offset_seconds
            label_name = moderation_label_info["ModerationLabel"]["Name"]
            confidence = moderation_label_info["ModerationLabel"]["Confidence"]

            moderation_label_dict['timestamp'] = timestamp
            moderation_label_dict['Moderated Label Name'] = label_name
            moderation_label_dict['Confidence'] = confidence

            result_list.append(moderation_label_dict)

            print('Timestamp: ', timestamp, '\tModerated Label: ', label_name, "\t Confidence: ", confidence)
        return result_list
    
    def GetCelebrityRekognitionResults(self):
        maxResults = 100
        paginationToken = ''
        # finished = False
        #
        # while finished == False:
        response = self.rek.get_celebrity_recognition(JobId=self.startJobId,
                                                   MaxResults=maxResults,
                                                   NextToken=paginationToken,
                                                   SortBy='TIMESTAMP')

        print('Codec: ' + response['VideoMetadata']['Codec'])
        # print('Duration: ' + str(response['VideoMetadata']['DurationMillis']))
        print('Format: ' + response['VideoMetadata']['Format'])
        print('Frame rate: ' + str(response['VideoMetadata']['FrameRate']))
        print()

        result_list = []

        for celeb_name in response['Celebrities']:
            celeb_dict = {}
            segment_offset_seconds = (self.segment - 1) * 60000
            timestamp = celeb_name["Timestamp"]
            timestamp += segment_offset_seconds
            name = celeb_name["Celebrity"]["Name"]
            confidence = celeb_name["Celebrity"]["Confidence"]

            celeb_dict['timestamp'] = timestamp
            celeb_dict['Celeb Name'] = name
            celeb_dict['Confidence'] = confidence

            result_list.append(celeb_dict)

            # print('Timestamp: ', timestamp, '\tModerated Label: ', label_name, "\t Confidence: ", confidence)
        return result_list


    def CreateTopicandQueue(self):

        millis = str(int(round(time.time() * 1000)))

        #Create SNS topic

        snsTopicName="AmazonRekognitionExample" + millis

        topicResponse=self.sns.create_topic(Name=snsTopicName)
        self.snsTopicArn = topicResponse['TopicArn']

        #create SQS queue
        sqsQueueName="AmazonRekognitionQueue" + millis
        self.sqs.create_queue(QueueName=sqsQueueName)
        self.sqsQueueUrl = self.sqs.get_queue_url(QueueName=sqsQueueName)['QueueUrl']

        attribs = self.sqs.get_queue_attributes(QueueUrl=self.sqsQueueUrl,
                                                AttributeNames=['QueueArn'])['Attributes']

        sqsQueueArn = attribs['QueueArn']

        # Subscribe SQS queue to SNS topic
        self.sns.subscribe(
            TopicArn=self.snsTopicArn,
            Protocol='sqs',
            Endpoint=sqsQueueArn)

        #Authorize SNS to write SQS queue
        policy = """{{
  "Version":"2012-10-17",
  "Statement":[
    {{
      "Sid":"MyPolicy",
      "Effect":"Allow",
      "Principal" : {{"AWS" : "*"}},
      "Action":"SQS:SendMessage",
      "Resource": "{}",
      "Condition":{{
        "ArnEquals":{{
          "aws:SourceArn": "{}"
        }}
      }}
    }}
  ]
}}""".format(sqsQueueArn, self.snsTopicArn)

        response = self.sqs.set_queue_attributes(
            QueueUrl = self.sqsQueueUrl,
            Attributes = {
                'Policy' : policy
            })

    def DeleteTopicandQueue(self):
        self.sqs.delete_queue(QueueUrl=self.sqsQueueUrl)
        self.sns.delete_topic(TopicArn=self.snsTopicArn)


def delete_temp_file(file_path):
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))


In [ ]:
class VideoDetect:

    jobId = ''

    rek = boto3.client('rekognition', region_name='ap-south-1')

    sqs = boto3.client('sqs', region_name='ap-south-1')

    sns = boto3.client('sns', region_name='ap-south-1')

 

    roleArn = ''

    bucket = ''

    video = ''

    startJobId = ''

 

    sqsQueueUrl = ''

    snsTopicArn = ''

    processType = ''

 

    def __init__(self, role, bucket, video, segment):

        self.roleArn = role

        self.bucket = bucket

        self.video = video

        self.segment = segment

 

    def GetSQSMessageSuccess(self):

 

        jobFound = False

        succeeded = False

 

        dotLine=0

        while jobFound == False:

            sqsResponse = self.sqs.receive_message(QueueUrl=self.sqsQueueUrl, MessageAttributeNames=['ALL'],

                                                   MaxNumberOfMessages=10)

 

            if sqsResponse:

 

                if 'Messages' not in sqsResponse:

                    if dotLine<40:

                        print('.', end='')

                        dotLine=dotLine+1

                    else:

                        print()

                        dotLine=0

                    sys.stdout.flush()

                    time.sleep(5)

                    continue

 

                for message in sqsResponse['Messages']:

                    notification = json.loads(message['Body'])

                    rekMessage = json.loads(notification['Message'])

                    print(rekMessage['JobId'])

                    print(rekMessage['Status'])

                    if rekMessage['JobId'] == self.startJobId:

                        # print('Matching Job Found:' + rekMessage['JobId'])

                        jobFound = True

                        if (rekMessage['Status']=='SUCCEEDED'):

                            succeeded=True

 

                        self.sqs.delete_message(QueueUrl=self.sqsQueueUrl,

                                                ReceiptHandle=message['ReceiptHandle'])

                    else:

                        pass

                        # print("Job didn't match:" +

                        #       str(rekMessage['JobId']) + ' : ' + self.startJobId)

                    # Delete the unknown message. Consider sending to dead letter queue

                    self.sqs.delete_message(QueueUrl=self.sqsQueueUrl,

                                            ReceiptHandle=message['ReceiptHandle'])

 

 

        return succeeded

 

    # def StartCelebrityRecognition(self):

    def StartContentModeration(self):

        # response=self.rek.start_label_detection(Video={'S3Object': {'Bucket': self.bucket, 'Name': self.video}},

        #                                         NotificationChannel={'RoleArn': self.roleArn, 'SNSTopicArn': self.snsTopicArn})

        # response=self.rek.start_celebrity_recognition(Video={'S3Object': {'Bucket': self.bucket, 'Name': self.video}},

        print(self.bucket)

        print(self.video)

        response=self.rek.start_content_moderation(Video={'S3Object': {'Bucket': self.bucket, 'Name': self.video}},

                                                   NotificationChannel={'RoleArn': self.roleArn, 'SNSTopicArn': self.snsTopicArn})

 

        self.startJobId=response['JobId']

        # print('Start Job Id: ' + self.startJobId)

 

    # def GetCelebrityRecognitionResults(self):

    def GetContentModerationResults(self):

        maxResults = 100

        paginationToken = ''

        # finished = False

        #

        # while finished == False:

        response = self.rek.get_content_moderation(JobId=self.startJobId,

                                                   MaxResults=maxResults,

                                                   NextToken=paginationToken,

                                                   SortBy='TIMESTAMP')

 

        print('Codec: ' + response['VideoMetadata']['Codec'])

        # print('Duration: ' + str(response['VideoMetadata']['DurationMillis']))

        print('Format: ' + response['VideoMetadata']['Format'])

        print('Frame rate: ' + str(response['VideoMetadata']['FrameRate']))

        print()

 

        result_list = []

 

        for moderation_label_info in response['ModerationLabels']:

            moderation_label_dict = {}

            segment_offset_seconds = (self.segment - 1) * 60000

            timestamp = moderation_label_info["Timestamp"]

            timestamp += segment_offset_seconds

            label_name = moderation_label_info["ModerationLabel"]["Name"]

            confidence = moderation_label_info["ModerationLabel"]["Confidence"]

 

            milliseconds = timestamp

            seconds, milliseconds = divmod(milliseconds, 1000)

            minutes, seconds = divmod(seconds, 60)

            hours, minutes = divmod(minutes, 60)

            millisecond_223344= f"{hours:02d}:{minutes:02d}:{seconds:02d}"

 

            moderation_label_dict['timestamp_in_Hours_min_sec_format']=millisecond_223344

            moderation_label_dict['timestamp'] = timestamp

            moderation_label_dict['Moderated Label Name'] = label_name

            moderation_label_dict['Confidence'] = confidence

 

            result_list.append(moderation_label_dict)

 

            print('Timestamp: ', timestamp, '\tModerated Label: ', label_name, "\t Confidence: ", confidence)

        return result_list

 

    def CreateTopicandQueue(self):

 

        millis = str(int(round(time.time() * 1000)))

 

        #Create SNS topic

 

        snsTopicName="AmazonRekognitionExample" + millis

 

        topicResponse=self.sns.create_topic(Name=snsTopicName)

        self.snsTopicArn = topicResponse['TopicArn']

 

        #create SQS queue

        sqsQueueName="AmazonRekognitionQueue" + millis

        self.sqs.create_queue(QueueName=sqsQueueName)

        self.sqsQueueUrl = self.sqs.get_queue_url(QueueName=sqsQueueName)['QueueUrl']

 

        attribs = self.sqs.get_queue_attributes(QueueUrl=self.sqsQueueUrl,

                                                AttributeNames=['QueueArn'])['Attributes']

 

        sqsQueueArn = attribs['QueueArn']

 

        # Subscribe SQS queue to SNS topic

        self.sns.subscribe(

            TopicArn=self.snsTopicArn,

            Protocol='sqs',

            Endpoint=sqsQueueArn)

 

        #Authorize SNS to write SQS queue

        policy = """{{

  "Version":"2012-10-17",

  "Statement":[

    {{

      "Sid":"MyPolicy",

      "Effect":"Allow",

      "Principal" : {{"AWS" : "*"}},

      "Action":"SQS:SendMessage",

      "Resource": "{}",

      "Condition":{{

        "ArnEquals":{{

          "aws:SourceArn": "{}"

        }}

      }}

    }}

  ]

}}""".format(sqsQueueArn, self.snsTopicArn)

 

        response = self.sqs.set_queue_attributes(

            QueueUrl = self.sqsQueueUrl,

            Attributes = {

                'Policy' : policy

            })

 

    def DeleteTopicandQueue(self):

        self.sqs.delete_queue(QueueUrl=self.sqsQueueUrl)

        self.sns.delete_topic(TopicArn=self.snsTopicArn)

/

In [ ]:
def fun_yolo_custom_model(classes, model_path, video_path):
    model = torch.hub.load('ultralytics/yolov5', 'custom', path = model_path)
    # video_path = 'test.mp4'
    cap = cv2.VideoCapture(video_path)

    frame_rate = cap.get(cv2.CAP_PROP_FPS)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    second_results = []

    for second in range(int(frame_count / frame_rate)):
        start_frame = int(second * frame_rate)
        end_frame = int((second + 1) * frame_rate) - 1

        total_confidence = 0
        monument_names = set()

        for frame_idx in range(start_frame, end_frame + 1):
            ret, frame = cap.read()
            if not ret:
                break

            # Convert the frame to PIL Image
            image = Image.fromarray(frame)

            # Perform detection on the frame
            results = model(image)

            for nested_result_list in results.xywhn:
                for result_tensor in nested_result_list:
                    class_index = int(result_tensor[-1])
                    class_probability = float(result_tensor[-2])
                    monument_name = classes[class_index]
                    
                    total_confidence += class_probability
                    monument_names.add(monument_name)

        avg_confidence = total_confidence / frame_rate
        timestamp = time.strftime("%H:%M:%S", time.gmtime(second))
        monument_names_str = ', '.join(monument_names)

        if monument_names:
            second_results.append({
                'timestamp': timestamp,
                'monument_names': monument_names_str,
                'confidence': avg_confidence
            })
    
    cap.release()

    return second_results


In [ ]:
def convert_video_to_wav(video_path):
    video_data = mediainfo(video_path)
    channels = video_data["channels"]
    bit_rate = video_data["bit_rate"]
    sample_rate = video_data["sample_rate"]
    wav_output = video_path.replace(".mp4",".wav").replace("video","wav_file")

    ffmpeg_cmd = ['ffmpeg', '-i', video_path,  '-b:a', bit_rate, '-ar', sample_rate , '-ac', channels ,'-vn', wav_output]
    
    try:
        subprocess.check_output(ffmpeg_cmd, stderr=subprocess.STDOUT)
        print("Video converted to WAV successfully!")
        return wav_output
    except subprocess.CalledProcessError as e:
        print("Error converting video to WAV:", e.output)
        return None


In [ ]:
# def transcribe(key,region,lang,path_in,path_out="out.txt",newLine=False):auto_detect_source_language_config
def transcribe(key,region,auto_detect_source_language_config,path_in,path_out="out.txt",newLine=False):
    # lang = auto_detect_source_language_config
    speech_config = speechsdk.SpeechConfig(subscription=key, region=region)
    # speech_config.speech_recognition_language=lang
    speech_config.output_format = speechsdk.OutputFormat(1)
    speech_config.request_word_level_timestamps()
    audio_config = speechsdk.AudioConfig(filename=path_in)
    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config,auto_detect_source_language_config=auto_detect_source_language_config, audio_config=audio_config)
    done = False
    tep_text_json = []
    textOut = ""
    def stop_cb(evt):
        print(1)
        print(evt)
        speech_recognizer.stop_continuous_recognition()
        nonlocal done
        done = True

    str_newLine = ""
    if newLine:
        str_newLine = " \n"

    def outPrint(evt):
        nonlocal textOut
        nonlocal tep_text_json
        tmp_text = evt.result.text
        tmp_timestamp = evt.result.offset
        tmp_duration_ts = evt.result.duration
        textOut += f"[{tmp_timestamp}] [{tmp_duration_ts}] {tmp_text}" + str_newLine
        
        out = eval(evt.result.json)
        tep_text_json.append(out)
        # print(2)
        # print(tmp_text)
        # print(3)
        # print(tmp_timestamp)
        # print(4)
        # print(tmp_duration_ts)
        # print(5)
        # print('-' * 100)
        # print(evt)
        # print('-' * 100)
        # print(evt.result)
        # print('-' * 100)
        # print("haa yahi h be")
        # print(evt.result.json)
        # out = eval(evt.result.json)
        # print(type(out))
        # print(out)
        # print('-' * 100)
        # print(evt.result.properties)
        # print('-' * 100)
        # print(dir(evt))
        # print('-' * 100)
        # print(dir(evt.result))
    
    speech_recognizer.recognized.connect(outPrint)
    speech_recognizer.session_stopped.connect(stop_cb)
    speech_recognizer.start_continuous_recognition()

    while not done:
        time.sleep(.5)
    # with open(path_out, 'w', encoding='utf-16') as f:
    #     f.write(textOut)
    # tep_text_json = str(tep_text_json)
    # path_out="./"
    print("tep_text_json:",tep_text_json)
    json_data = json.dumps(tep_text_json)
    print("json_data:",json_data)
    with open(path_out, 'w', encoding='utf-16') as f:
        f.write(json_data)

In [ ]:
def milliseconds_to_srt_timestamp(milliseconds):
    total_seconds = milliseconds // 1000
    # milliseconds %= 1000
    # minutes = seconds // 60
    # seconds %= 60
    hours = total_seconds // 3600
    minutes = (total_seconds % 3600) // 60
    seconds = total_seconds % 60
    milliseconds = milliseconds % 1000
    # minutes %= 60
    return '{:02d}:{:02d}:{:02d},{:03d}'.format(hours, minutes, seconds, milliseconds)


# json_file = "D:/Python_scripts/speech_to_test_azure/New folder/txt_&_json_file/txt_&_json_file/Piyush_2.json"  # Replace with the path to your JSON file
def convert_json_into_words_by_words_list(json_file):
    with open(json_file, "r",encoding='utf-16') as file:
        json_output = file.read()

    data = json.loads(json_output)
    # print(len(data))
    srt_content = ""
    list_of_words = []
    for j in range(len(data)):
        # print(j)
    # Extract word-level details
        words = data[j]["NBest"][0]["Words"]

    # Generate SRT content
        for i, word in enumerate(words):
            list_with_time =[]
            start_timestamp = milliseconds_to_srt_timestamp(word["Offset"] // 10000)
            end_timestamp = milliseconds_to_srt_timestamp((word["Offset"]  + word["Duration"]) // 10000)
            subtitle_text = word["Word"]
            srt_entry = f"{i+1}\n{start_timestamp} --> {end_timestamp}\n{subtitle_text}\n\n"
            srt_content += srt_entry
            list_with_time.append(start_timestamp)
            list_with_time.append(end_timestamp)
            list_with_time.append(subtitle_text)
            list_of_words.append(list_with_time)
    json_data = json.dumps(list_of_words)
    with open("./list_of_words.json","w",encoding="utf-16") as f:
        f.write(json_data)
    return list_of_words

# json_file = "D:/Python_scripts/speech_to_test_azure/srt_fol/pm_modi.json"

# list_of_words = convert_json_into_words_by_words_list(json_file)

In [ ]:

def convert_time_stamp_to_seconds(time_stamp):
    time_format = "%H:%M:%S,%f"

    # Convert the time stamp to a datetime object
    time_obj = datetime.strptime(time_stamp, time_format)

    # Calculate the total number of seconds
    seconds = time_obj.hour * 3600 + time_obj.minute * 60 + time_obj.second + time_obj.microsecond / 1000000

    return seconds

In [ ]:
def generate_srt_file(list_of_words,filename):

    list_of_new_srt = []
    str_of_words = ""
    total_words_covered = 0
    index = 1
    srt_sentence = ""

    list_for_sentences_for_the_final_srt =[]
    while total_words_covered != len(list_of_words):
        transcription_list = []
        start_point = convert_time_stamp_to_seconds(list_of_words[total_words_covered][0])
        start_for_srt = list_of_words[total_words_covered][0]

        transcription_list.append(list_of_words[total_words_covered][2])
        total_words_covered += 1

        
        list_for_srt_by_sentence = []
        
        dict = {}

        # DECIDE AN END FOR START WORD

        for j in range(total_words_covered, (len(list_of_words) - 1)):

            end_point =convert_time_stamp_to_seconds(list_of_words[j][1])
            end_for_srt = list_of_words[j][1]
            end_for_prev_srt = list_of_words[j - 1][1]
        
            time_limit = round(((end_point) - (start_point)),3)
            
            
            # bin_size
            if time_limit <= (2.50):
                transcription_list.append(list_of_words[j][2])
                total_words_covered += 1
                end_for_srt = list_of_words[j][1]

            else:
                list_for_srt_by_sentence.append(transcription_list)
                dict["transcript"] = transcription_list
                dict["start_point"] = start_for_srt
                dict["end_point"]  = end_for_prev_srt
                break
    

        list_of_new_srt.append(dict)
        str_of_words = ""
    for transcript_dict in list_of_new_srt:
        dict ={}
        if "transcript" in transcript_dict.keys():
            subtitle_text = (' '.join(transcript_dict["transcript"]))
            start_time = transcript_dict["start_point"]
            end_time = transcript_dict["end_point"] 
            dict["transcript"] = subtitle_text
            dict["start_point"] = start_time
            dict["end_point"] = end_time
            list_for_sentences_for_the_final_srt.append(dict)

    for dict_sentences in list_for_sentences_for_the_final_srt:
        # print(dict_sentences)
        subtitle_text = dict_sentences["transcript"]
        start_timestamp = dict_sentences['start_point']
        end_timestamp = dict_sentences['end_point']
        srt_entry = f"{index}\n{start_timestamp} --> {end_timestamp}\n{subtitle_text}\n\n"
        index +=1
        # print(srt_entry)
        # break
        srt_sentence += srt_entry

        # srt_content += ""+ srt_entry

    filename = filename

    with open(filename, 'w') as file:
        file.write(srt_sentence)
    return srt_sentence


In [3]:
from moviepy.editor import VideoFileClip


def split_video_into_segments(input_file, segment_duration=60):
    try:
        clip = VideoFileClip(input_file)

        # Calculate the total duration of the input video in seconds
        total_duration = clip.duration

        # Calculate the number of segments required
        num_segments = int(total_duration / segment_duration)

        # Extract the file name and extension from the input_file
        file_name, file_extension = input_file.split('.')

        # Split the video into segments
        for i in range(num_segments):
            start_time = i * segment_duration
            end_time = min((i + 1) * segment_duration, total_duration)
            segment_clip = clip.subclip(start_time, end_time)
            output_file = f"{file_name}_segment_{i + 1}.{file_extension}"
            segment_clip.write_videofile(output_file, codec="libx264")

        # For the last segment (if it's shorter than segment_duration)
        if total_duration % segment_duration > 0:
            start_time = num_segments * segment_duration
            end_time = total_duration
            last_segment_clip = clip.subclip(start_time, end_time)
            output_file = f"{file_name}_segment_{num_segments + 1}.{file_extension}"
            last_segment_clip.write_videofile(output_file, codec="libx264")

        clip.close()
        print("Video splitting completed successfully!")
    except Exception as e:
        print(f"An error occurred: {str(e)}")

In [4]:
def split_video_and_upload_s3(video_path):
    cwd = os.getcwd()
    listdir_earlier = os.listdir(cwd)
    print(listdir_earlier)
    # ffmpeg_cmd = ['ffmpeg', '-i', video_path,  '-c', 'copy', '-map 0', '-segment_time' , '00:01:00', "-f segment -reset_timestamps 1 output%03d.mp4"]
    # ffmpeg_split_cmd = """ffmpeg -i '""" + video_path + """' -c copy -map 0 -segment_time 00:01:00 -f segment -reset_timestamps 1 output%03d.mp4"""
    ffmpeg_split_cmd = """ffmpeg -i 'video/THOMAS_SHELBY_SMOKING.mp4' -c copy -map 0 -segment_time 00:01:00 -f segment -reset_timestamps 1 output%03d.mp4 > /dump 2>&1"""
    # subprocess.check_output(ffmpeg_cmd, stderr=subprocess.STDOUT)
    # subprocess.run(ffmpeg_split_cmd, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    try:
        print(ffmpeg_split_cmd)
        subprocess.check_output(ffmpeg_split_cmd, shell=True, stderr=subprocess.STDOUT)
    except subprocess.CalledProcessError as e:
        raise RuntimeError("command '{}' return with error (code {}): {}".format(e.cmd, e.returncode, e.output))

    listdir_now = os.listdir(cwd)
    print(listdir_now)

    # cmd = 'aws s3 cp D:/Python_scripts/AWS_Scripts/my_function/my_deployment_package.zip s3://alt-cap-logsbucket-12leb47kv5fyr/'

In [5]:
def upload_to_s3(video_path):
    cmd = 'aws s3 cp "' + video_path + '" s3://aws-rekognition-dataset-v2'
    # subprocess.run(cmd, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    try:
        subprocess.check_output(cmd, shell=True, stderr=subprocess.STDOUT)
    except subprocess.CalledProcessError as e:
        raise RuntimeError("command '{}' return with error (code {}): {}".format(e.cmd, e.returncode, e.output))
    

In [6]:
def get_videos_with_nomenclature():
    eligible_videos = []
    for filename in os.listdir('D:/Demo_Resources/Moderation_AI/video'):
        if filename.endswith('.mp4') and '_segment_' in filename:
            filename = 'D:/Demo_Resources/Moderation_AI/video/' + str(filename)
            eligible_videos.append(filename)
            upload_to_s3(filename)  # Upload the video to S3 (you need to implement this function)
    return eligible_videos

In [7]:
import re

def extract_segment_number(filename):
    pattern = r'_segment_(\d+)\.mp4'
    match = re.search(pattern, filename)
    if match:
        segment_number = int(match.group(1))
        return segment_number
    else:
        return None

In [ ]:
from flask import Flask, request, jsonify
import os
import shutil
from moviepy.editor import VideoFileClip
    
app = Flask(__name__)

@app.route('/upload_video', methods=['POST'])
def upload_video():
    # try: 
        save_path = 'D:/Demo_Resources/Moderation_AI/video/'
        # json_path = 'D:/Demo_Resources/Transcription_AI/Transcript_flask/json_file/'
        key = os.get("Azure_key")
        region = "centralindia"
        
        auto_detect_source_language_config = speechsdk.languageconfig.AutoDetectSourceLanguageConfig(
        languages=["en-IN", "hi-IN",])
        
        # lang = "en-IN" # See e.g. https://learn.microsoft.com/en-us/dynamics365/fin-ops-core/dev-itpro/help/language-locale
        # path_in = "09-02-2023 LS-01.wav"
        # path_in = "D:/Python_scripts/speech_to_test_azure/New folder/pm_modi.wav"
        # path_out = "D:/Python_scripts/speech_to_test_azure/srt_fol/pm_modi.json"
        # transcribe(key,region,lang,path_in,path_out)
        # Check if the 'video' key exists in the request files
        if 'video' not in request.files:
            return 'No video file found', 400

        video_file = request.files['video']

        # Save the video to the specified path
        video_file.save(save_path + video_file.filename)
        path_of_video = (save_path + video_file.filename)
        path_of_wav_file = convert_video_to_wav(path_of_video)
        # path_of_wav_file = convert_video_to_wav(path_of_video)
        
        # prefix = os.path.dirname(path_of_video)
        # print(prefix)
        # print(type(prefix))
        
        # FOR SRT FROM AZURE CONTINUOUS SPEECH
        path_of_json = (json_path + video_file.filename)
        path_of_json = path_of_json.replace(".mp4",".json")
        transcribe(key,region,auto_detect_source_language_config,path_of_wav_file,path_of_json)

        
        list_of_words = convert_json_into_words_by_words_list(path_of_json)
        path_of_srt = path_of_video.replace("video","srt_file").replace(".mp4",".srt")
        output_value_of_srt = generate_srt_file(list_of_words,path_of_srt)

        split_video_into_segments(path_of_video)
        list_video_segments = get_videos_with_nomenclature()

        final_output = []

        for video_segment in list_video_segments:
        # upload_to_s3(path_of_video)

            roleArn = 'arn:aws:iam::Account_ID:role/AmazonRekognitionVideoServiceRole'
            bucket = 'your_s3_bucket'
            # UNCOMMENT THIS FOR SHORT VIDEO
            # video = video_file.filename
            video = video_segment
            video = video.split('/')[-1]
            segment = extract_segment_number(video)

            print('Content Moderation Summary:')

            analyzer = VideoDetect(roleArn, bucket, video, segment)
            analyzer.CreateTopicandQueue()

            output_value_of_srt = None

        # analyzer.StartCelebrityRecognition()
            analyzer.StartContentModeration()
            for i in range(1):
                if analyzer.GetSQSMessageSuccess()==True:
                    # analyzer.GetCelebrityRecognitionResults()
                    output_value_of_srt = analyzer.GetContentModerationResults()
                    print('Right before break!!')
                    final_output.append(output_value_of_srt)
                    break
            analyzer.DeleteTopicandQueue()
        for video_segment in list_video_segments:
            delete_temp_file(video_segment)
        return final_output, 200

if __name__ == '__main__':
    app.run()
    # print("="*100)
    # print(a)


In [ ]:
%pip install moviepy